In [1]:
%load_ext rpy2.ipython

In [25]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)
library(dplyr)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"

GTEx_DATA_DIR <- IN_DIR
GTEx_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GTEx_DATA_DIR,GTEx_DATA_FIL,sep=""))

In [ ]:
%%R

variances <- apply(assay(final_result), 1, var)

zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data_20.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
library(tibble)

keep_samples <- colData(final_result)$dose %in% c(0.00, 30.00)

final_result <- final_result[, keep_samples]

temp_df <- as_tibble(colData(final_result)) %>%
  filter(!grepl("SRP131784", study))

tcdd_data <- temp_df
final_result <- final_result[, colData(final_result)$external_id %in% temp_df$external_id]

print(dim(final_result))
print(dim(tcdd_data))


[1] 43252   136
[1] 136 198


In [ ]:
%%R
library(dplyr)
library(SummarizedExperiment)

sampling_percentage <- 0.20

tcdd_data_dose0 <- filter(tcdd_data, dose == 0)
tcdd_data_dose30 <- filter(tcdd_data, dose == 30)

min_count <- min(nrow(tcdd_data_dose0), nrow(tcdd_data_dose30)) * sampling_percentage
min_count <- floor(min_count)  

sampled_tcdd_dose0 <- sample_n(tcdd_data_dose0, size = min_count)
sampled_tcdd_dose30 <- sample_n(tcdd_data_dose30, size = min_count)

tcdd_data <- bind_rows(sampled_tcdd_dose0, sampled_tcdd_dose30)

final_result <- final_result[, colData(final_result)$external_id %in% tcdd_data$external_id]
matched_indices <- match(tcdd_data$external_id, colData(final_result)$external_id)
final_result <- final_result[, matched_indices]

print(dim(tcdd_data))
print(dim(final_result))


[1]  26 198
[1] 43252    26


In [ ]:
%%R
dose_counts <- table(tcdd_data$dose)

print(dose_counts)

dose_counts_df <- as.data.frame(dose_counts)

colnames(dose_counts_df) <- c("Dose", "Count")

write.csv(dose_counts_df, paste0(OUT_DIR, "dose_counts030_time_course_20.csv"), row.names=FALSE)


 0 30 
13 13 


In [30]:
%%R
combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
print(combined_gender)

 [1] "female" "male"   "male"   "male"   "male"   "male"   "male"   "female"
 [9] "female" "male"   "male"   "male"   "male"   "male"   "female" "male"  
[17] "male"   "male"   "male"   "male"   "male"   "male"   "male"   "male"  
[25] "male"   "female"


In [31]:
%%R
write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id030_time_course_20.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender030_time_course_20.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id030_time_course_20.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose030_time_course_20.txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
saveRDS(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose_detail_vec030_time_course_20.Rds",sep=""))
saveRDS(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_detail_vec030_time_course_20.Rds",sep=""))


In [32]:
%%R
tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()
print(tcdd.df)

                            V1       V2      V3      V4      V5      V6
ENSMUSG00000079800.2        50        0       0       0       0       0
ENSMUSG00000095092.1         0        0       0       0       0       0
ENSMUSG00000079192.2         0        0       0       0       0       0
ENSMUSG00000079794.2       100        0       0       0       0       0
ENSMUSG00000094799.1         0        0       0       0       0       0
ENSMUSG00000095250.1         0        0       0       0       0       0
ENSMUSG00000095787.1         0        0       0       0       0       0
ENSMUSG00000096100.1         0        0       0       0      50       0
ENSMUSG00000094054.1       200        0       0       0       0       0
ENSMUSG00000095672.1        50        0       0       0       0       0
ENSMUSG00000079190.3       248        0       0       0       0     150
ENSMUSG00000094514.1         0        0       0       0       0       0
ENSMUSG00000094915.1         0        0       0       0       0 

ENSMUSG00000103884.1         0        0       0       0       0       0
ENSMUSG00000084353.1     10314     2213    1763    1650    2509    2448
ENSMUSG00000103933.1     22856    29283   23201   42426   43947   34721
ENSMUSG00000076135.1        43        0       8       0       0       0
ENSMUSG00000086195.1       203       50       6       8      48       0
ENSMUSG00000104504.1        97        0      18       0       7      18
ENSMUSG00000066693.2     10788     4026    1635    2836    4881    4694
ENSMUSG00000102316.1         0        0       0       0       0       0
ENSMUSG00000103819.1         0        0       0       0       0       0
ENSMUSG00000102871.1         0        0       0       0       0       0
ENSMUSG00000025909.16      121        0     100     149       0     193
ENSMUSG00000101571.2       200       50     150     100     250     300
ENSMUSG00000104428.1         0        0       0       0       0       0
ENSMUSG00000102272.1      4058     2543    1830    1546    3917 

ENSMUSG00000091020.3      6742     2296    1446    3843    2394    2149
ENSMUSG00000101640.1         0        0       0       0       0       0
ENSMUSG00000042686.5      1543      750     450     200    2496     448
ENSMUSG00000025777.8         0        0       0       0       0       0
ENSMUSG00000099899.1         0        0       0       0       0       0
ENSMUSG00000100398.1       448      498     200     246     544     297
ENSMUSG00000101589.1      6518     2823    1140    2690    2339    2535
ENSMUSG00000102048.1       250      349     150     250     350     150
ENSMUSG00000067780.3        98        0      50       0       0       0
ENSMUSG00000100053.1         0        0       0       0       0       0
ENSMUSG00000085125.7         0        0       0       0       0       0
ENSMUSG00000025776.13        0        0       0       0       0       0
ENSMUSG00000099895.1         0        0       0       0       0       0
ENSMUSG00000100554.1      1431     1096     566     350    1293 

ENSMUSG00000080391.1         0        0       0       0       0       0
ENSMUSG00000099492.1     64003    16161    8033   17910   15497   15958
ENSMUSG00000101726.1         0        0      50       0       0       0
ENSMUSG00000100261.1     10262     2682    1987    3124    4439    3151
ENSMUSG00000101180.1         0        0       0       0       0       0
ENSMUSG00000103783.1         0        0       0       0       0       0
ENSMUSG00000104315.1         0        0       0      50     100       0
ENSMUSG00000048874.15    64408    33947   19858   29721   40416   30458
ENSMUSG00000026064.16    93916    45504   38770   60659   81297   36393
ENSMUSG00000117310.2     92986    44765   37776   59346   80377   35701
ENSMUSG00000117091.1         0        0       0       0       0       0
ENSMUSG00000086726.3         0        0       0       0       0       0
ENSMUSG00000100237.1        50        0       0       0      99       0
ENSMUSG00000100910.1         0        0       0       0       0 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

    11076
ENSMUSG00000040596.15    11697    12566
ENSMUSG00000084093.1      1724     7390
ENSMUSG00000051081.7         0        0
ENSMUSG00000085841.1         0        0
ENSMUSG00000086925.1         0        0
ENSMUSG00000102548.1        50        0
ENSMUSG00000097823.2       199      150
ENSMUSG00000060568.14      150      200
ENSMUSG00000102870.1        50        0
ENSMUSG00000103333.1        49        0
ENSMUSG00000103139.1         0        0
ENSMUSG00000026558.13    11306    10403
ENSMUSG00000102909.1         0        0
ENSMUSG00000052428.10   116507   111124
ENSMUSG00000088246.1       829     1258
ENSMUSG00000095752.1         0        0
ENSMUSG00000026687.14   136057   149836
ENSMUSG00000026688.5     26473    27086
ENSMUSG00000015843.10     4184     3679
ENSMUSG00000026686.14        0        0
ENSMUSG00000093538.2         0        0
ENSMUSG00000103780.1         0        0
ENSMUSG00000102594.1        50      100
ENSMUSG00000052534.15     5156    16558
ENSMUSG00000103463.1         0

ENSMUSG00000026523.14        0        0
ENSMUSG00000039748.11      545      246
ENSMUSG00000104158.2         0        0
ENSMUSG00000078184.1      4473     6347
ENSMUSG00000103277.1         0        0
ENSMUSG00000055214.15        0        0
ENSMUSG00000102446.1         0        0
ENSMUSG00000104248.1         0        0
ENSMUSG00000087875.1         0        0
ENSMUSG00000103634.1         0        0
ENSMUSG00000104154.1         0      150
ENSMUSG00000103435.1         0        0
ENSMUSG00000065573.1         0        0
ENSMUSG00000084749.1         0        0
ENSMUSG00000057335.11     6629     8696
ENSMUSG00000103367.1        90      103
ENSMUSG00000103584.1        39      100
ENSMUSG00000102475.1         0        0
ENSMUSG00000102211.1        50      172
ENSMUSG00000087082.1         0        0
ENSMUSG00000103251.1         0        0
ENSMUSG00000026504.17     3247     4636
ENSMUSG00000104391.1         0       50
ENSMUSG00000102133.1         0        0
ENSMUSG00000102749.1         0        0


ENSMUSG00000102546.1       950     1658
ENSMUSG00000103056.1         0       38
ENSMUSG00000039246.8     12495    10538
ENSMUSG00000102217.1         0        0
ENSMUSG00000103690.1       493      320
ENSMUSG00000089045.1         0        0
ENSMUSG00000039239.14    14262    18954
ENSMUSG00000100393.1        32      174
ENSMUSG00000102619.1        50        0
ENSMUSG00000001305.5      7017     7706
ENSMUSG00000097784.1       575      958
ENSMUSG00000103503.1         0        0
ENSMUSG00000097935.1       100       50
ENSMUSG00000039224.1      1085     1344
ENSMUSG00000104424.1         0        0
ENSMUSG00000089831.1         0        0
ENSMUSG00000106839.1         0        0
ENSMUSG00000107163.1         0        0
ENSMUSG00000089947.1         0        0
ENSMUSG00000089730.2         0        0
ENSMUSG00000026611.17      100       50
ENSMUSG00000104394.2      1345     2004
ENSMUSG00000039210.16    11366    13455
ENSMUSG00000104245.1       725     1520
ENSMUSG00000103923.1        42      100


ENSMUSG00000092190.1      1193     1207
ENSMUSG00000019794.13    10495    11439
ENSMUSG00000085211.2       397     1222
ENSMUSG00000040006.13    23138    30306
ENSMUSG00000081497.1        50       50
ENSMUSG00000015757.10    13025    15195
ENSMUSG00000093060.1         0        0
ENSMUSG00000088857.1         0        0
ENSMUSG00000039981.6      2192     4290
ENSMUSG00000015755.16    49504    60380
ENSMUSG00000112360.1         0        0
ENSMUSG00000112109.1        50      601
ENSMUSG00000047712.8       599      539
ENSMUSG00000111215.1         0        0
ENSMUSG00000065692.1         0        0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [33]:
%%R
print(dim(tcdd.df))
setdiff(rownames(tcdd.df), rownames(final_result))

In [34]:
%%R
colnames(tcdd.df) <- colData(final_result)$external_id
print(tcdd.df)

     665622     106090     159333     184985     201114
ENSMUSG00000005681.12   12779704    2917430    1868287    2027466    3716702
ENSMUSG00000058715.11      24205       2759       1096      12165       2489
ENSMUSG00000013593.12     284699     117939      90026      88543     147157
ENSMUSG00000006403.13        750        150         50        400         50
ENSMUSG00000052423.14      25429       7374       3867       8324       8958
ENSMUSG00000062729.11      55802      21318      14531      17627      18058
ENSMUSG00000053483.13      30375      10346       5045      11570      11233
ENSMUSG00000062963.10      70553      33509      22439      32154      38411
ENSMUSG00000079137.3       28523      18432      13977      15579      22124
ENSMUSG00000103983.1        2783       2343        699       1748       2397
ENSMUSG00000013973.12      54740      23610      14465      23118      25372
ENSMUSG00000013997.10     181537      69696      52013      67493      76567
ENSMUSG00000102929.1

ENSMUSG00000090272.9       14509       4456       2192       9204       5286
ENSMUSG00000026536.9        6209       1173        343       2786       1282
ENSMUSG00000039997.16      16460       6236       1846      10575       4158
ENSMUSG00000026535.9           0          0          0         50          0
ENSMUSG00000103588.1           0          0          0         49          0
ENSMUSG00000054203.7        5075       1116        300       3765       1011
ENSMUSG00000045381.6           0          0          0         50          0
ENSMUSG00000102603.2           0          0          0          0          0
ENSMUSG00000047048.4           0          0          0          0          0
ENSMUSG00000050134.7           0          0          0          0          0
ENSMUSG00000059371.4           0          0          0          0          0
ENSMUSG00000046486.4           0          0          0          0          0
ENSMUSG00000102371.4           0          0          0          0          0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



          0          0          0          0          0
ENSMUSG00000060487.7        1499        600        346        692        200
ENSMUSG00000111901.1           0          0          0          0          0
ENSMUSG00000112200.1           0          0          0          0          0
ENSMUSG00000064609.1           0          0          0          0          0
ENSMUSG00000019790.17      20554      12606       6802      10997      10401
ENSMUSG00000112146.1           0          0          0          0          0
ENSMUSG00000112059.1           0          0          0          0          0
ENSMUSG00000112074.1           0          0          0          0          0
ENSMUSG00000065876.1           0          0          0          0          0
ENSMUSG00000112713.1           0          0          0          0          0
ENSMUSG00000112060.1           0          0          0          0          0
ENSMUSG00000108973.1           0          0          0          0          0
ENSMUSG00000101621.2

ENSMUSG00000019851.8      121545      68710      41445      62073      75362
ENSMUSG00000019850.11       3699       1239       1288      20467        530
ENSMUSG00000111821.1           0          0          0          0          0
ENSMUSG00000110770.1        1899        300        334        644        524
ENSMUSG00000111052.1           0          0          0          0          0
ENSMUSG00000111606.1         100          0         50         50         99
ENSMUSG00000110917.1           0          0          0          0          0
ENSMUSG00000045591.6           0          0          0          0          0
ENSMUSG00000110902.1           0          0          0         50          0
ENSMUSG00000110835.1           0          0          0          0          0
ENSMUSG00000020009.13      33198       5972       4997      13558      10577
ENSMUSG00000039760.8          50          0          0          0          0
ENSMUSG00000110958.1           0          0          0          0          0

...


ENSMUSG00000093650.2           0
ENSMUSG00000066842.18       1586
ENSMUSG00000099622.1           0
ENSMUSG00000096851.1           0
ENSMUSG00000102628.1           0
ENSMUSG00000100595.1           0
ENSMUSG00000097426.1         538
ENSMUSG00000102095.1         190
ENSMUSG00000114212.1           0
ENSMUSG00000114943.1           0
ENSMUSG00000114591.1           0
ENSMUSG00000088498.1           0
ENSMUSG00000100954.4        1156
ENSMUSG00000023150.14      85056
ENSMUSG00000052748.14       8288
ENSMUSG00000106406.1           0
ENSMUSG00000103928.1        4023
ENSMUSG00000053286.11      16739
ENSMUSG00000103238.1           0
ENSMUSG00000026484.13      11528
ENSMUSG00000100981.1           0
ENSMUSG00000100551.1        5721
ENSMUSG00000026483.13       5115
ENSMUSG00000100594.1        4942
ENSMUSG00000043019.12      35320
ENSMUSG00000084557.1           0
ENSMUSG00000100685.1         150
ENSMUSG00000032666.16       3741
ENSMUSG00000102090.1           0
ENSMUSG00000014980.14       3474
ENSMUSG00

ENSMUSG00000026586.16       1223
ENSMUSG00000088951.1           0
ENSMUSG00000040124.5        4166
ENSMUSG00000102117.1        7084
ENSMUSG00000089700.1           0
ENSMUSG00000026585.13       4570
ENSMUSG00000026584.14      25446
ENSMUSG00000041406.14       4118
ENSMUSG00000041396.13       3340
ENSMUSG00000026582.6         200
ENSMUSG00000026581.14       6468
ENSMUSG00000090081.1         100
ENSMUSG00000026580.16       5585
ENSMUSG00000086593.2        5709
ENSMUSG00000089790.1           0
ENSMUSG00000026579.8      407475
ENSMUSG00000040918.12      64132
ENSMUSG00000026578.6        5181
ENSMUSG00000026577.13       9239
ENSMUSG00000103493.1          87
ENSMUSG00000026575.15       9556
ENSMUSG00000026576.12      36747
ENSMUSG00000104011.1         308
ENSMUSG00000103226.1           0
ENSMUSG00000097281.2           0
ENSMUSG00000102389.1          50
ENSMUSG00000103383.1           0
ENSMUSG00000102897.1           0
ENSMUSG00000102163.1           0
ENSMUSG00000102761.1           0
ENSMUSG000

ENSMUSG00000073491.11       2198
ENSMUSG00000106299.1           0
ENSMUSG00000043263.13       2832
ENSMUSG00000066677.12        100
ENSMUSG00000103105.5          50
ENSMUSG00000103051.1           0
ENSMUSG00000073490.11      12225
ENSMUSG00000073489.6       12683
ENSMUSG00000090222.2        2987
ENSMUSG00000102433.1           0
ENSMUSG00000090272.9       12910
ENSMUSG00000026536.9        5237
ENSMUSG00000039997.16      14192
ENSMUSG00000026535.9         197
ENSMUSG00000103588.1          49
ENSMUSG00000054203.7        4415
ENSMUSG00000045381.6           0
ENSMUSG00000102603.2           0
ENSMUSG00000047048.4           0
ENSMUSG00000050134.7           0
ENSMUSG00000059371.4           0
ENSMUSG00000046486.4           0
ENSMUSG00000102371.4           0
ENSMUSG00000091950.3           0
ENSMUSG00000055033.4           0
ENSMUSG00000026532.7         200
ENSMUSG00000102920.1           0
ENSMUSG00000051509.6           0
ENSMUSG00000066671.4           0
ENSMUSG00000028354.13       9153
ENSMUSG000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



       1678
ENSMUSG00000112255.1         250
ENSMUSG00000037151.8       10125
ENSMUSG00000101792.1           0
ENSMUSG00000020090.7           0
ENSMUSG00000020089.8       64503
ENSMUSG00000112581.1           0
ENSMUSG00000020088.10      52951
ENSMUSG00000020087.5       24098
ENSMUSG00000020085.15      14462
ENSMUSG00000020086.6        1804
ENSMUSG00000112574.1         100
ENSMUSG00000058806.14       4718
ENSMUSG00000112018.1           0
ENSMUSG00000046687.6      557588
ENSMUSG00000044312.4           0
ENSMUSG00000020083.13       6139
ENSMUSG00000099185.1           0
ENSMUSG00000037031.10       5660
ENSMUSG00000020081.5           0
ENSMUSG00000085347.1          50
ENSMUSG00000037012.18       4441
ENSMUSG00000020080.8         699
ENSMUSG00000020079.15      14687
ENSMUSG00000097129.2         549
ENSMUSG00000020078.16      30688
ENSMUSG00000020077.14      14398
ENSMUSG00000036955.16       6696
ENSMUSG00000105203.1         199
ENSMUSG00000020075.8       46574
ENSMUSG00000112347.1         29

In [35]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
# print(ensembl_wo_ids)

In [36]:
%%R
deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
print(length(deleted_ensembl_wo_ids))

[1] 12169


In [37]:
%%R
rownames(tcdd.df) <- ensembl_wo_ids
# print(rownames(tcdd.df))

In [38]:
%%R
rownames(deleted_data) <- deleted_ensembl_wo_ids
print(rownames(deleted_data))

    [1] "ENSMUSG00000096776" "ENSMUSG00000096244" "ENSMUSG00000095450"
    [4] "ENSMUSG00000089163" "ENSMUSG00000099278" "ENSMUSG00000094121"
    [7] "ENSMUSG00000079764" "ENSMUSG00000079774" "ENSMUSG00000094741"
   [10] "ENSMUSG00000074720" "ENSMUSG00000094337" "ENSMUSG00000095570"
   [13] "ENSMUSG00000098647" "ENSMUSG00000094431" "ENSMUSG00000095247"
   [16] "ENSMUSG00000095585" "ENSMUSG00000096873" "ENSMUSG00000095755"
   [19] "ENSMUSG00000095207" "ENSMUSG00000093828" "ENSMUSG00000095623"
   [22] "ENSMUSG00000095666" "ENSMUSG00000096680" "ENSMUSG00000096236"
   [25] "ENSMUSG00000096756" "ENSMUSG00000094722" "ENSMUSG00000095728"
   [28] "ENSMUSG00000094836" "ENSMUSG00000094383" "ENSMUSG00000094474"
   [31] "ENSMUSG00000096550" "ENSMUSG00000094172" "ENSMUSG00000091585"
   [34] "ENSMUSG00000095763" "ENSMUSG00000095523" "ENSMUSG00000095475"
   [37] "ENSMUSG00000095019" "ENSMUSG00000102693" "ENSMUSG00000064842"
   [40] "ENSMUSG00000102851" "ENSMUSG00000103377" "ENSMUSG00000102348"
   [43

  [346] "ENSMUSG00000104104" "ENSMUSG00000087883" "ENSMUSG00000099892"
  [349] "ENSMUSG00000100977" "ENSMUSG00000077039" "ENSMUSG00000100684"
  [352] "ENSMUSG00000099770" "ENSMUSG00000088510" "ENSMUSG00000101890"
  [355] "ENSMUSG00000087908" "ENSMUSG00000088594" "ENSMUSG00000100955"
  [358] "ENSMUSG00000099469" "ENSMUSG00000101297" "ENSMUSG00000100059"
  [361] "ENSMUSG00000088161" "ENSMUSG00000100935" "ENSMUSG00000103991"
  [364] "ENSMUSG00000084704" "ENSMUSG00000103060" "ENSMUSG00000089866"
  [367] "ENSMUSG00000089206" "ENSMUSG00000084228" "ENSMUSG00000044594"
  [370] "ENSMUSG00000058017" "ENSMUSG00000080729" "ENSMUSG00000073602"
  [373] "ENSMUSG00000073601" "ENSMUSG00000087210" "ENSMUSG00000089278"
  [376] "ENSMUSG00000077237" "ENSMUSG00000099484" "ENSMUSG00000104478"
  [379] "ENSMUSG00000101231" "ENSMUSG00000100884" "ENSMUSG00000089037"
  [382] "ENSMUSG00000084741" "ENSMUSG00000100290" "ENSMUSG00000100758"
  [385] "ENSMUSG00000102728" "ENSMUSG00000088691" "ENSMUSG00000099884"
  [388

  [691] "ENSMUSG00000077749" "ENSMUSG00000111753" "ENSMUSG00000088423"
  [694] "ENSMUSG00000044499" "ENSMUSG00000103299" "ENSMUSG00000111549"
  [697] "ENSMUSG00000096080" "ENSMUSG00000077576" "ENSMUSG00000111234"
  [700] "ENSMUSG00000088925" "ENSMUSG00000111048" "ENSMUSG00000111208"
  [703] "ENSMUSG00000110782" "ENSMUSG00000094521" "ENSMUSG00000094311"
  [706] "ENSMUSG00000111183" "ENSMUSG00000019834" "ENSMUSG00000111098"
  [709] "ENSMUSG00000110889" "ENSMUSG00000046922" "ENSMUSG00000111576"
  [712] "ENSMUSG00000094663" "ENSMUSG00000111315" "ENSMUSG00000093298"
  [715] "ENSMUSG00000087891" "ENSMUSG00000099893" "ENSMUSG00000098642"
  [718] "ENSMUSG00000076282" "ENSMUSG00000110891" "ENSMUSG00000077082"
  [721] "ENSMUSG00000098418" "ENSMUSG00000077591" "ENSMUSG00000098993"
  [724] "ENSMUSG00000098409" "ENSMUSG00000111182" "ENSMUSG00000094681"
  [727] "ENSMUSG00000065718" "ENSMUSG00000111499" "ENSMUSG00000111419"
  [730] "ENSMUSG00000110843" "ENSMUSG00000111719" "ENSMUSG00000112062"
  [733

 [1036] "ENSMUSG00000107400" "ENSMUSG00000094734" "ENSMUSG00000108114"
 [1039] "ENSMUSG00000058513" "ENSMUSG00000088147" "ENSMUSG00000096858"
 [1042] "ENSMUSG00000094702" "ENSMUSG00000071065" "ENSMUSG00000050478"
 [1045] "ENSMUSG00000069421" "ENSMUSG00000063173" "ENSMUSG00000049052"
 [1048] "ENSMUSG00000052818" "ENSMUSG00000061961" "ENSMUSG00000107399"
 [1051] "ENSMUSG00000058071" "ENSMUSG00000094295" "ENSMUSG00000095608"
 [1054] "ENSMUSG00000107423" "ENSMUSG00000048745" "ENSMUSG00000044897"
 [1057] "ENSMUSG00000108253" "ENSMUSG00000095804" "ENSMUSG00000112688"
 [1060] "ENSMUSG00000059862" "ENSMUSG00000045559" "ENSMUSG00000088352"
 [1063] "ENSMUSG00000085559" "ENSMUSG00000085512" "ENSMUSG00000064632"
 [1066] "ENSMUSG00000077304" "ENSMUSG00000088688" "ENSMUSG00000080833"
 [1069] "ENSMUSG00000086808" "ENSMUSG00000082897" "ENSMUSG00000098853"
 [1072] "ENSMUSG00000084318" "ENSMUSG00000089247" "ENSMUSG00000104137"
 [1075] "ENSMUSG00000088861" "ENSMUSG00000064766" "ENSMUSG00000087990"
 [1078

 [1381] "ENSMUSG00000086617" "ENSMUSG00000084200" "ENSMUSG00000064788"
 [1384] "ENSMUSG00000000125" "ENSMUSG00000080694" "ENSMUSG00000089479"
 [1387] "ENSMUSG00000086089" "ENSMUSG00000087184" "ENSMUSG00000088279"
 [1390] "ENSMUSG00000098989" "ENSMUSG00000064691" "ENSMUSG00000087139"
 [1393] "ENSMUSG00000085324" "ENSMUSG00000020624" "ENSMUSG00000086760"
 [1396] "ENSMUSG00000086135" "ENSMUSG00000078600" "ENSMUSG00000075431"
 [1399] "ENSMUSG00000089460" "ENSMUSG00000084130" "ENSMUSG00000064588"
 [1402] "ENSMUSG00000081318" "ENSMUSG00000078586" "ENSMUSG00000086588"
 [1405] "ENSMUSG00000085336" "ENSMUSG00000110653" "ENSMUSG00000082562"
 [1408] "ENSMUSG00000085059" "ENSMUSG00000084887" "ENSMUSG00000080616"
 [1411] "ENSMUSG00000083155" "ENSMUSG00000099690" "ENSMUSG00000085970"
 [1414] "ENSMUSG00000112240" "ENSMUSG00000092168" "ENSMUSG00000112723"
 [1417] "ENSMUSG00000112559" "ENSMUSG00000088209" "ENSMUSG00000102096"
 [1420] "ENSMUSG00000099360" "ENSMUSG00000077259" "ENSMUSG00000112787"
 [1423

...

 [8944] "ENSMUSG00000063206" "ENSMUSG00000082859" "ENSMUSG00000095813"
 [8947] "ENSMUSG00000095507" "ENSMUSG00000096340" "ENSMUSG00000084254"
 [8950] "ENSMUSG00000082698" "ENSMUSG00000058568" "ENSMUSG00000006570"
 [8953] "ENSMUSG00000044743" "ENSMUSG00000047390" "ENSMUSG00000045337"
 [8956] "ENSMUSG00000048500" "ENSMUSG00000110681" "ENSMUSG00000044222"
 [8959] "ENSMUSG00000017049" "ENSMUSG00000089484" "ENSMUSG00000106605"
 [8962] "ENSMUSG00000083285" "ENSMUSG00000106471" "ENSMUSG00000070084"
 [8965] "ENSMUSG00000077399" "ENSMUSG00000109413" "ENSMUSG00000109158"
 [8968] "ENSMUSG00000053979" "ENSMUSG00000109299" "ENSMUSG00000089302"
 [8971] "ENSMUSG00000088736" "ENSMUSG00000110399" "ENSMUSG00000105292"
 [8974] "ENSMUSG00000118460" "ENSMUSG00000087981" "ENSMUSG00000110075"
 [8977] "ENSMUSG00000110417" "ENSMUSG00000110383" "ENSMUSG00000109689"
 [8980] "ENSMUSG00000110297" "ENSMUSG00000031491" "ENSMUSG00000065367"
 [8983] "ENSMUSG00000063932" "ENSMUSG00000104743" "ENSMUSG00000074404"
 [8986

 [9289] "ENSMUSG00000095453" "ENSMUSG00000095723" "ENSMUSG00000064664"
 [9292] "ENSMUSG00000080516" "ENSMUSG00000110437" "ENSMUSG00000088710"
 [9295] "ENSMUSG00000099326" "ENSMUSG00000087870" "ENSMUSG00000077538"
 [9298] "ENSMUSG00000077771" "ENSMUSG00000078840" "ENSMUSG00000091028"
 [9301] "ENSMUSG00000096519" "ENSMUSG00000074564" "ENSMUSG00000096736"
 [9304] "ENSMUSG00000096201" "ENSMUSG00000106183" "ENSMUSG00000110425"
 [9307] "ENSMUSG00000098373" "ENSMUSG00000084690" "ENSMUSG00000110927"
 [9310] "ENSMUSG00000065382" "ENSMUSG00000111353" "ENSMUSG00000084747"
 [9313] "ENSMUSG00000088076" "ENSMUSG00000092776" "ENSMUSG00000098408"
 [9316] "ENSMUSG00000093959" "ENSMUSG00000096815" "ENSMUSG00000109590"
 [9319] "ENSMUSG00000111568" "ENSMUSG00000110951" "ENSMUSG00000111691"
 [9322] "ENSMUSG00000110734" "ENSMUSG00000110858" "ENSMUSG00000111458"
 [9325] "ENSMUSG00000111503" "ENSMUSG00000089477" "ENSMUSG00000111830"
 [9328] "ENSMUSG00000111285" "ENSMUSG00000111479" "ENSMUSG00000111555"
 [9331

 [9634] "ENSMUSG00000091147" "ENSMUSG00000110809" "ENSMUSG00000111309"
 [9637] "ENSMUSG00000110923" "ENSMUSG00000102020" "ENSMUSG00000111638"
 [9640] "ENSMUSG00000111633" "ENSMUSG00000111267" "ENSMUSG00000111510"
 [9643] "ENSMUSG00000111717" "ENSMUSG00000098935" "ENSMUSG00000098429"
 [9646] "ENSMUSG00000097241" "ENSMUSG00000103536" "ENSMUSG00000110748"
 [9649] "ENSMUSG00000077415" "ENSMUSG00000111697" "ENSMUSG00000099266"
 [9652] "ENSMUSG00000089066" "ENSMUSG00000088760" "ENSMUSG00000098420"
 [9655] "ENSMUSG00000099816" "ENSMUSG00000077666" "ENSMUSG00000111245"
 [9658] "ENSMUSG00000065122" "ENSMUSG00000111770" "ENSMUSG00000110783"
 [9661] "ENSMUSG00000101673" "ENSMUSG00000111281" "ENSMUSG00000111822"
 [9664] "ENSMUSG00000111663" "ENSMUSG00000065274" "ENSMUSG00000110956"
 [9667] "ENSMUSG00000111386" "ENSMUSG00000065706" "ENSMUSG00000110946"
 [9670] "ENSMUSG00000111418" "ENSMUSG00000103891" "ENSMUSG00000111874"
 [9673] "ENSMUSG00000047897" "ENSMUSG00000099539" "ENSMUSG00000102684"
 [9676

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000101383"
[11830] "ENSMUSG00000099914" "ENSMUSG00000099369" "ENSMUSG00000101893"
[11833] "ENSMUSG00000100442" "ENSMUSG00000101518" "ENSMUSG00000101374"
[11836] "ENSMUSG00000101255" "ENSMUSG00000101713" "ENSMUSG00000099531"
[11839] "ENSMUSG00000100008" "ENSMUSG00000100537" "ENSMUSG00000101134"
[11842] "ENSMUSG00000100535" "ENSMUSG00000099738" "ENSMUSG00000101988"
[11845] "ENSMUSG00000101336" "ENSMUSG00000101593" "ENSMUSG00000101444"
[11848] "ENSMUSG00000099733" "ENSMUSG00000101339" "ENSMUSG00000099731"
[11851] "ENSMUSG00000096902" "ENSMUSG00000100694" "ENSMUSG00000101844"
[11854] "ENSMUSG00000100898" "ENSMUSG00000101857" "ENSMUSG00000101836"
[11857] "ENSMUSG00000102008" "ENSMUSG00000100869" "ENSMUSG00000101277"
[11860] "ENSMUSG00000099371" "ENSMUSG00000102129" "ENSMUSG00000100359"
[11863] "ENSMUSG00000099495" "ENSMUSG00000101747" "ENSMUSG00000100379"
[11866] "ENSMUSG00000101715" "ENSMUSG00000101425" "ENSMUSG00000100698"
[11869] "ENSMUSG00000099547" "ENSMUSG00000099782" "ENSM

In [39]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(reactome_ensembl_ids)

   [1] "ENSMUSG00000000001" "ENSMUSG00000000028" "ENSMUSG00000000049"
   [4] "ENSMUSG00000000058" "ENSMUSG00000000085" "ENSMUSG00000000088"
   [7] "ENSMUSG00000000120" "ENSMUSG00000000126" "ENSMUSG00000000127"
  [10] "ENSMUSG00000000142" "ENSMUSG00000000149" "ENSMUSG00000000154"
  [13] "ENSMUSG00000000168" "ENSMUSG00000000171" "ENSMUSG00000000182"
  [16] "ENSMUSG00000000183" "ENSMUSG00000000184" "ENSMUSG00000000197"
  [19] "ENSMUSG00000000214" "ENSMUSG00000000215" "ENSMUSG00000000216"
  [22] "ENSMUSG00000000223" "ENSMUSG00000000247" "ENSMUSG00000000253"
  [25] "ENSMUSG00000000263" "ENSMUSG00000000275" "ENSMUSG00000000276"
  [28] "ENSMUSG00000000290" "ENSMUSG00000000296" "ENSMUSG00000000301"
  [31] "ENSMUSG00000000303" "ENSMUSG00000000305" "ENSMUSG00000000308"
  [34] "ENSMUSG00000000320" "ENSMUSG00000000326" "ENSMUSG00000000340"
  [37] "ENSMUSG00000000374" "ENSMUSG00000000394" "ENSMUSG00000000399"
  [40] "ENSMUSG00000000409" "ENSMUSG00000000420" "ENSMUSG00000000435"
  [43] "ENSMUSG00000

In [40]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids030_20.Rds",sep=""))

In [41]:
%%R
tcdd.df <- tcdd.df[reactome_ensembl_ids,]
print(tcdd.df)

     170750      45871      29916      58053      56786
ENSMUSG00000000276       4454       2298        550       1548       1778
ENSMUSG00000000290      13826       1396       1378      12679       1608
ENSMUSG00000000296      13862       3386       4029       4374       6960
ENSMUSG00000000301     521798     169674     120287     184561     212098
ENSMUSG00000000303      36828      38280      12657      24561      25870
ENSMUSG00000000305       1042        224        296        100        392
ENSMUSG00000000308        192         22          0          0          0
ENSMUSG00000000320       1197        198        149        240        551
ENSMUSG00000000326     740711     522388     269932     260601     417227
ENSMUSG00000000340      84487      66566      37717      63289      75775
ENSMUSG00000000374      42422      20686      11643      19084      18685
ENSMUSG00000000394          0          0          0          0          0
ENSMUSG00000000399     161428      63940      50103     

ENSMUSG00000004231          0          0          0          0          0
ENSMUSG00000004263      53489      12412       9902      13928      17982
ENSMUSG00000004264     232189     116152      79271     110686     131607
ENSMUSG00000004266      42029      12684       7444      20491       9660
ENSMUSG00000004267        210         50         50         86         50
ENSMUSG00000004268      83695      28678      20189      32478      33670
ENSMUSG00000004270     225971      72803      58403      52644      94753
ENSMUSG00000004285      51837      14667       9229      16495      17760
ENSMUSG00000004296          0          0          0        100          0
ENSMUSG00000004317       8848       3920       2933       4870       3879
ENSMUSG00000004319      66605      30392      21669      39863      42505
ENSMUSG00000004328         40        100          0        200        200
ENSMUSG00000004341       1330       1248        100        637        937
ENSMUSG00000004344         89        1

ENSMUSG00000005370      19549       8922       4596       7486       9419
ENSMUSG00000005371      37720      18622      12788      16370      19918
ENSMUSG00000005373     662197     314857     155548     236112     337245
ENSMUSG00000005378      54313      24410      12198      20992      17619
ENSMUSG00000005397      28279       5567       1632       6660       4031
ENSMUSG00000005410       3563       1592        250       2240       1098
ENSMUSG00000005413      41356       8570       4424      23602       8353
ENSMUSG00000005447       2392       1597       1513       2024       2509
ENSMUSG00000005465       2866        400        200       1448        550
ENSMUSG00000005469     117134      35512      14103      21842      23456
ENSMUSG00000005474         50          0          0          0          0
ENSMUSG00000005481      22887      14043       5280      17970      12221
ENSMUSG00000005483      23221      15733       5584      19194      20002
ENSMUSG00000005493          0         

ENSMUSG00000006522    1386694     574096     340809    1283670     500426
ENSMUSG00000006538        950       2246        745        476       3506
ENSMUSG00000006542        512        100        173        451        622
ENSMUSG00000006567      56526      18553      13987      16607      24690
ENSMUSG00000006574        100         50         50        244        100
ENSMUSG00000006576       1144        354        199        550        700
ENSMUSG00000006585       5646       2237       1770       2448       1847
ENSMUSG00000006589      43357      16129      13327      20375      18989
ENSMUSG00000006599      30313      14037      12077      20522      20886
ENSMUSG00000006611      78579      34075      16323      35451      27107
ENSMUSG00000006641      10777       2519      10378       3590      12843
ENSMUSG00000006649        525          0          0          0          0
ENSMUSG00000006676     142822      49374      33731      48037      57627
ENSMUSG00000006678       3914       16

ENSMUSG00000009108        585        291        138        845        497
ENSMUSG00000009246        253          0          0         53          3
ENSMUSG00000009281     936769     165842     155147     182332     264308
ENSMUSG00000009292      14150       2484        984      16194       3642
ENSMUSG00000009293      58305      24258      10755      38644      17932
ENSMUSG00000009350        149          0          0       8038          0
ENSMUSG00000009356          0          0          0          0          0
ENSMUSG00000009376      84308      38978      42966      47489      69853
ENSMUSG00000009394        150          0          0          0         50
ENSMUSG00000009406      11495       3144       2937       3608       4907
ENSMUSG00000009470      77612      42711      34349      37711      52039
ENSMUSG00000009487          0          0          0          0          0
ENSMUSG00000009535      18927       6339       4582       7593       8249
ENSMUSG00000009545       1934        7

...

ENSMUSG00000025283      82295
ENSMUSG00000025287       4919
ENSMUSG00000025289      21499
ENSMUSG00000025290      82381
ENSMUSG00000025314      27640
ENSMUSG00000025316       2726
ENSMUSG00000025317      10943
ENSMUSG00000025321       1247
ENSMUSG00000025324       3280
ENSMUSG00000025326      52137
ENSMUSG00000025328        100
ENSMUSG00000025329        100
ENSMUSG00000025330        452
ENSMUSG00000025332      63241
ENSMUSG00000025333          0
ENSMUSG00000025340       9578
ENSMUSG00000025348       6999
ENSMUSG00000025350      18430
ENSMUSG00000025351      15481
ENSMUSG00000025353      21212
ENSMUSG00000025355      81451
ENSMUSG00000025357       5843
ENSMUSG00000025358       9752
ENSMUSG00000025362      28803
ENSMUSG00000025364      55841
ENSMUSG00000025366      15090
ENSMUSG00000025369      59772
ENSMUSG00000025370          0
ENSMUSG00000025371      12517
ENSMUSG00000025372      11226
ENSMUSG00000025373       7330
ENSMUSG00000025374      18949
ENSMUSG00000025380          0
ENSMUSG000

ENSMUSG00000026213      13375
ENSMUSG00000026219     123208
ENSMUSG00000026222      12855
ENSMUSG00000026228          0
ENSMUSG00000026229      87156
ENSMUSG00000026234     236867
ENSMUSG00000026235        598
ENSMUSG00000026237         50
ENSMUSG00000026239       3213
ENSMUSG00000026240      13797
ENSMUSG00000026241          0
ENSMUSG00000026246          0
ENSMUSG00000026248      13426
ENSMUSG00000026249       2796
ENSMUSG00000026251          0
ENSMUSG00000026253          0
ENSMUSG00000026254      26462
ENSMUSG00000026259       9987
ENSMUSG00000026260      90899
ENSMUSG00000026270      16870
ENSMUSG00000026271       1918
ENSMUSG00000026272      82321
ENSMUSG00000026276      76270
ENSMUSG00000026280      24139
ENSMUSG00000026281      13476
ENSMUSG00000026283       7617
ENSMUSG00000026285          0
ENSMUSG00000026288       7848
ENSMUSG00000026289      21213
ENSMUSG00000026295      57411
ENSMUSG00000026304       8991
ENSMUSG00000026307      55799
ENSMUSG00000026311       7331
ENSMUSG000

ENSMUSG00000026965      33073
ENSMUSG00000026966       8519
ENSMUSG00000026971       1768
ENSMUSG00000026976          0
ENSMUSG00000026981      19754
ENSMUSG00000026983          0
ENSMUSG00000026984          0
ENSMUSG00000026985          0
ENSMUSG00000026991      70956
ENSMUSG00000026994       5168
ENSMUSG00000026999       2465
ENSMUSG00000027002     101644
ENSMUSG00000027009      14189
ENSMUSG00000027010      17829
ENSMUSG00000027011      16800
ENSMUSG00000027012      33750
ENSMUSG00000027014       7438
ENSMUSG00000027015        600
ENSMUSG00000027018      10183
ENSMUSG00000027022          0
ENSMUSG00000027035      18062
ENSMUSG00000027048     248593
ENSMUSG00000027067      47347
ENSMUSG00000027068       1953
ENSMUSG00000027070        349
ENSMUSG00000027071       1848
ENSMUSG00000027072          0
ENSMUSG00000027073         99
ENSMUSG00000027075       1762
ENSMUSG00000027078      31181
ENSMUSG00000027079      14682
ENSMUSG00000027080      12297
ENSMUSG00000027082       6242
ENSMUSG000

ENSMUSG00000027883       7843
ENSMUSG00000027889      14281
ENSMUSG00000027890     161781
ENSMUSG00000027893      66846
ENSMUSG00000027895          0
ENSMUSG00000027901       9381
ENSMUSG00000027905      14677
ENSMUSG00000027907      12116
ENSMUSG00000027931      11499
ENSMUSG00000027933      18655
ENSMUSG00000027935       2745
ENSMUSG00000027938        409
ENSMUSG00000027940      51047
ENSMUSG00000027947      45985
ENSMUSG00000027950       6760
ENSMUSG00000027951      22092
ENSMUSG00000027952      15872
ENSMUSG00000027953       6612
ENSMUSG00000027954      18253
ENSMUSG00000027957      73724
ENSMUSG00000027962      15236
ENSMUSG00000027966          0
ENSMUSG00000027971          0
ENSMUSG00000027977         50
ENSMUSG00000027981      15375
ENSMUSG00000027983       7439
ENSMUSG00000027984     235232
ENSMUSG00000027985        489
ENSMUSG00000027994        283
ENSMUSG00000027995       3193
ENSMUSG00000027996         33
ENSMUSG00000027997      54761
ENSMUSG00000027998      15301
ENSMUSG000

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [42]:
%%R
saveRDS(tcdd.df,file=paste(OUT_DIR,"tcdd_df030_time_course_20.Rds",sep=""))

In [43]:
%%R
saveRDS(deleted_data, file=paste(OUT_DIR, "format_name_deleted_data030_time_course_20.Rds", sep=""))